In [1]:
import torch
from transformers import AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import evaluate

/home/maximo/miniconda3/envs/cnc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = AutoModelForSequenceClassification.from_pretrained("VCNC/Auto-CNC").to(device)

In [4]:
tokens = {"other":0, "1200":1, "1300":2, "1400":3, "1600":4, "1700":5, "1800":6, "1900":7, "2100":8, "2400":9, "2405":9, "2600":10, "2800":11, 
          "2805":11, "AR-10":12, "AR-20":12, "AR-30":12, "AR-40":12, "AR-50":12, "AR-70":12, "AR-90":12}

In [5]:
df = pd.read_hdf('data.h5', keys='df')
df

,PEDIDO,FE_EMIT,FE_ENTREGA,X,y
3503,56227,737304,737304,[0],[0]
3502,56226,737305,737305,[6],[6]
3501,56231,737306,737306,[6],[6]
3499,56238,737318,737318,"[11, 0]",[11]
3500,56244,737318,737318,"[11, 0]",[0]
...,...,...,...,...,...
4,60629,738513,738759,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]",[10]
3,60644,738513,738759,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]",[10]
2,60643,738513,738759,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]",[10]
1,60604,738513,738759,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]",[10]


In [6]:
#Misma lógica que al finetunear
df['X_tuple'] = df.X.apply(tuple)
df['y_tuple'] = df.y.apply(tuple)
df = df.drop_duplicates(subset=['X_tuple', 'y_tuple']) #Eliminamos los datos replicados X->y
df = df.drop(labels=['FE_EMIT', 'FE_ENTREGA', 'PEDIDO', 'X_tuple', 'y_tuple'], axis=1) #Eliminamos columnas que no son útiles
df['y'] = df['y'].apply(lambda x: x[0])
df

,X,y
3503,[0],0
3502,[6],6
3499,"[11, 0]",11
3500,"[11, 0]",0
3497,"[6, 11]",6
...,...,...
31,"[6, 6, 6, 6, 11, 11, 11, 0, 11, 11, 11, 11, 11...",0
22,"[11, 11, 11, 11, 11, 10, 10, 10, 10, 11, 11, 1...",11
17,"[11, 11, 11, 11, 11, 10, 10, 10, 10, 11, 11, 1...",10
10,"[11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 1...",11


In [7]:
#Definimos funciones que son necesarias para el input de BERT, consultese: https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt para más información
from typing import Union
#Función para padear las secuencias
def pad(max_len:int, sequence:Union[list, np.array], pad_token:int=21):
  padding = np.full(max_len-len(sequence), pad_token)
  padded_seq = np.concatenate((sequence, padding))
  return padded_seq.tolist()

#Función para crear las máscaras de atención
def attention_mask(pad_sequence:Union[list, np.array], pad_token:int=21):
  seq = np.asarray(pad_sequence, dtype=int)
  attention_mask = np.asarray(seq != pad_token, dtype=int)
  return attention_mask.tolist()

#Función para crear los arreglos que especifican el tipo de token
def token_types(pad_sequence:Union[list, np.array]):
  return np.zeros(len(pad_sequence), dtype=int).tolist()

In [8]:
#Preparamos el df para BERT con las columnas que requiere('idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids')
max_len = df.X.apply(lambda x: len(x))
max_len = max(max_len)
dt = df.copy(True)
PAD = 13

dt.insert(0, 'idx', np.full(len(df), np.nan))
dt.insert(1, 'label', dt.y)
dt.insert(2, 'input_ids', np.full(len(df), np.nan))
dt.insert(3, 'attention_mask', np.full(len(df), np.nan))
dt.insert(4, 'token_type_ids', np.full(len(df), np.nan))
data = dt.to_numpy()

for i in range(len(data)):
  data[i][2] = pad(max_len, data[i][5], pad_token=PAD)
  data[i][3] = attention_mask(data[i][2], pad_token=PAD)
  data[i][4] = token_types(data[i][2])
dt['input_ids'] = data[:, 2]
dt['attention_mask'] = data[:, 3]
dt['token_type_ids'] = data[:, 4]
df = dt.copy(True)
df

,idx,label,input_ids,attention_mask,token_type_ids,X,y
3503,NaN,0,"[0, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0],0
3502,NaN,6,"[6, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[6],6
3499,NaN,11,"[11, 0, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 0]",11
3500,NaN,0,"[11, 0, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 0]",0
3497,NaN,6,"[6, 11, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 11]",6
...,...,...,...,...,...,...,...
31,NaN,0,"[6, 6, 6, 6, 11, 11, 11, 0, 11, 11, 11, 11, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 6, 6, 6, 11, 11, 11, 0, 11, 11, 11, 11, 11...",0
22,NaN,11,"[11, 11, 11, 11, 11, 10, 10, 10, 10, 11, 11, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 11, 11, 11, 11, 10, 10, 10, 10, 11, 11, 1...",11
17,NaN,10,"[11, 11, 11, 11, 11, 10, 10, 10, 10, 11, 11, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 11, 11, 11, 11, 10, 10, 10, 10, 11, 11, 1...",10
10,NaN,11,"[11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 1...",11


In [9]:
data = df.to_numpy()
input_ids = np.asarray([[np.asarray(i, dtype=int) for i in data[:, 2]]])
input_ids = torch.tensor(input_ids).to(device)

attention_mask = np.asarray([[np.asarray(i, dtype=int) for i in data[:, 3]]])
attention_mask = torch.tensor(attention_mask).to(device)

token_type_ids = np.asarray([[np.asarray(i, dtype=int) for i in data[:, 4]]])
token_type_ids = torch.tensor(token_type_ids).to(device)

In [ ]:
outputs = []
for i in range(len(input_ids[0])):
    outputs.append(model(input_ids=input_ids[:, i], attention_mask=attention_mask[:, i], token_type_ids=token_type_ids[:, i]))
    torch.cuda.empty_cache()